In [1]:
!nvidia-smi

Tue Sep 26 12:54:10 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.99                 Driver Version: 536.99       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   47C    P0              15W /  50W |      0MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import tensorflow as tf
from helper_functions import *

In [3]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [5]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [6]:
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [7]:
print(f"Total training samples: {len(train_df)}")
print(f"Total test samples: {len(test_df)}")
print(f"Total samples: {len(train_df) + len(test_df)}")

Total training samples: 7613
Total test samples: 3263
Total samples: 10876


In [8]:
import random
random_index = random.randint(0, len(train_df)-5) 
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
    _, text, target = row
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n{text}\n")
    print("---\n")

Target: 1 (real disaster)
Text:
OUN cancels Severe Thunderstorm Warning for Lincoln Logan Oklahoma [OK]  http://t.co/bTi8JAMFiu #WX

---

Target: 1 (real disaster)
Text:
Remembering Mordechai Yehuda Friedman 24 of Ramat Beit Shemesh; murdered by Hamas terrorists in the suicide bombing of Egged bus No. 361

---

Target: 0 (not real disaster)
Text:
Somebody get the doctor I'm feelin' pretty poor. Somebody get the stretcher
before I hit the floor

---

Target: 1 (real disaster)
Text:
@rinkydnk2 @ZaibatsuNews @NeoProgressive1 When push2Left talk='ecology'&amp;'human rts'&amp;'democracy'. War Afghetc='Left' humanitarian bombing

---

Target: 0 (not real disaster)
Text:
He came to a land which was engulfed in tribal war and turned it into a land of peace i.e. Madinah. #ProphetMuhammad #islam

---



In [9]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [10]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [11]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

In [12]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,
                                    output_mode="int",
                                    output_sequence_length=None)

In [13]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [14]:
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [15]:
text_vectorizer.adapt(train_sentences)

In [16]:
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int64)>

In [17]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence} \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
Football hooligan jailed for rioting before game in Scotland was already banned from matches in England #UkNews http://t.co/q5mp2Q6Hy8 

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1523,    1, 5327,   10,  367,  239,  397,    4, 3437,   23,  659,
         793,   20, 3651,    4]], dtype=int64)>

In [18]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}") 
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [19]:
words_in_vocab

['',
 '[UNK]',
 'the',
 'a',
 'in',
 'to',
 'of',
 'and',
 'i',
 'is',
 'for',
 'on',
 'you',
 'my',
 'with',
 'it',
 'that',
 'at',
 'by',
 'this',
 'from',
 'be',
 'are',
 'was',
 'have',
 'like',
 'as',
 'up',
 'so',
 'just',
 'but',
 'me',
 'im',
 'your',
 'not',
 'amp',
 'out',
 'its',
 'will',
 'an',
 'no',
 'has',
 'fire',
 'after',
 'all',
 'when',
 'we',
 'if',
 'now',
 'via',
 'new',
 'more',
 'get',
 'or',
 'about',
 'what',
 'he',
 'people',
 'news',
 'been',
 'over',
 'one',
 'how',
 'dont',
 'they',
 'who',
 'into',
 'were',
 'do',
 'us',
 '2',
 'can',
 'video',
 'emergency',
 'there',
 'disaster',
 'than',
 'police',
 'would',
 'his',
 'still',
 'her',
 'some',
 'body',
 'storm',
 'crash',
 'burning',
 'suicide',
 'back',
 'man',
 'california',
 'why',
 'time',
 'them',
 'had',
 'buildings',
 'rt',
 'first',
 'cant',
 'see',
 'got',
 'day',
 'off',
 'our',
 'going',
 'nuclear',
 'know',
 'world',
 'bomb',
 'fires',
 'love',
 'killed',
 'go',
 'attack',
 'youtube',
 'dead

In [20]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer="uniform",
                             input_length=max_length,
                             name="embedding_1") 

embedding

In [21]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence} \n\nEmbedded version:")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
Report: Corey Brewer was drug tested after 51-point explosion http://t.co/HhlLQCkcEP 

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.00564884, -0.00748349,  0.01101966, ...,  0.03134537,
          0.04849913, -0.00921624],
        [ 0.01387974, -0.00216342,  0.02028277, ...,  0.03333303,
          0.01617399,  0.02459632],
        [ 0.03977952, -0.03782602, -0.03646283, ...,  0.00236253,
          0.03332629,  0.02803668],
        ...,
        [ 0.01645621, -0.00589932, -0.01471175, ..., -0.02511839,
          0.00912381, -0.00024097],
        [ 0.01645621, -0.00589932, -0.01471175, ..., -0.02511839,
          0.00912381, -0.00024097],
        [ 0.01645621, -0.00589932, -0.01471175, ..., -0.02511839,
          0.00912381, -0.00024097]]], dtype=float32)>

In [22]:
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-0.00564884, -0.00748349,  0.01101966,  0.00216369,  0.03479229,
        0.01977248, -0.02031909,  0.01342938,  0.04059104, -0.0258708 ,
       -0.02306321,  0.02142851, -0.0116911 , -0.04733119,  0.02096704,
       -0.03517903,  0.00968939,  0.00644717,  0.00597665, -0.0446418 ,
        0.04844128, -0.00970294,  0.04908172,  0.0134231 , -0.02431868,
       -0.02838971,  0.03797746,  0.04082144, -0.02619135,  0.01592822,
        0.00081626, -0.02601033, -0.01299198,  0.03806952, -0.04328567,
        0.03325546,  0.03431619,  0.03487357, -0.01898768,  0.00096408,
        0.04788505, -0.01395931, -0.01699395, -0.01524813, -0.01852945,
       -0.02475393, -0.04050721, -0.01742535,  0.02547351,  0.00212555,
        0.02505534, -0.01936264, -0.02746921, -0.03983289, -0.01289146,
       -0.00477264, -0.04761383,  0.04139854, -0.02678755, -0.04115716,
        0.01146024,  0.04483441,  0.02772   ,  0.04895402, -0.02069873,
       -0.019946

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf", MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [24]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [25]:
train_df['target'].value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [26]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,

In [27]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
    model_accuracy = accuracy_score(y_true, y_pred) * 100
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    model_results = {"accuracy": model_accuracy,
                     "precision": model_precision,
                     "recall": model_recall,
                     "f1": model_f1}
    return model_results

In [28]:
baseline_results = calculate_results(y_true=val_labels, y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [29]:
from helper_functions import create_tensorboard_callback
SAVE_DIR = "model_logs"

In [30]:
from tensorflow.keras import layers, Model
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_1 = Model(inputs, outputs, name = 'model_1_dense')

In [31]:
model_1.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

In [32]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [33]:
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     'simple_dense_model')])

Saving TensorBoard log files to: model_logs/simple_dense_model/20230926-125427
Epoch 1/5
215/215 [==============================] - 4s 8ms/step - loss: 0.6094 - accuracy: 0.6916 - val_loss: 0.5357 - val_accuracy: 0.7572
Epoch 2/5
215/215 [==============================] - 1s 7ms/step - loss: 0.4410 - accuracy: 0.8189 - val_loss: 0.4691 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.3463 - accuracy: 0.8605 - val_loss: 0.4590 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2848 - accuracy: 0.8923 - val_loss: 0.4641 - val_accuracy: 0.7927
Epoch 5/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2380 - accuracy: 0.9118 - val_loss: 0.4767 - val_accuracy: 0.7874


In [34]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 4ms/step - loss: 0.4767 - accuracy: 0.7874


[0.47668465971946716, 0.787401556968689]

In [35]:
embedding.weights

[<tf.Variable 'embedding_1/embeddings:0' shape=(10000, 128) dtype=float32, numpy=
 array([[ 0.00073165,  0.01504799, -0.03425454, ..., -0.04403539,
         -0.01042282,  0.0187644 ],
        [ 0.04135863, -0.03945084, -0.03811941, ...,  0.00464736,
          0.03163552,  0.02928299],
        [ 0.00684032,  0.05363132, -0.00241555, ..., -0.07082179,
         -0.04750703,  0.01448253],
        ...,
        [-0.03301444, -0.0052493 , -0.04209725, ...,  0.02028764,
          0.00308807,  0.02215792],
        [ 0.00692343,  0.05942352, -0.01975194, ..., -0.06199061,
         -0.01018393,  0.03510419],
        [-0.0372346 ,  0.06267188, -0.07451148, ..., -0.02367218,
         -0.08643329,  0.01742155]], dtype=float32)>]

In [36]:
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [37]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs

24/24 [==============================] - 0s 4ms/step


array([[4.04882103e-01],
       [7.44331181e-01],
       [9.97895002e-01],
       [1.08899988e-01],
       [1.11435294e-01],
       [9.35560942e-01],
       [9.13459361e-01],
       [9.92534518e-01],
       [9.71568167e-01],
       [2.65703410e-01],
       [1.23554133e-01],
       [7.14031875e-01],
       [5.59769906e-02],
       [2.21097901e-01],
       [5.30175632e-03],
       [1.64252639e-01],
       [2.66677570e-02],
       [1.05061978e-01],
       [2.43335843e-01],
       [5.50095379e-01],
       [9.07310009e-01],
       [4.14080769e-02],
       [4.44304526e-01],
       [1.02827892e-01],
       [9.57856238e-01],
       [9.98957157e-01],
       [3.35513242e-02],
       [7.10333958e-02],
       [2.77729668e-02],
       [2.13070989e-01],
       [5.81133366e-01],
       [2.41698146e-01],
       [5.10923028e-01],
       [2.07991764e-01],
       [5.23500443e-01],
       [5.54889441e-02],
       [9.94535685e-01],
       [1.44162700e-01],
       [3.44177224e-02],
       [9.98458385e-01],


In [38]:
model_1_pred_probs.shape

(762, 1)

In [39]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1.,
       0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [40]:
model_1_results = calculate_results(y_true=val_labels, y_pred=model_1_preds)
model_1_results

{'accuracy': 78.74015748031496,
 'precision': 0.7914920592553047,
 'recall': 0.7874015748031497,
 'f1': 0.7846966492209201}

In [41]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

In [42]:
def compare_baseline_to_new_results(baseline_results, new_model_results):
    for key, value in baseline_results.items():
        print(f"Baseline {key}: {value:.2f}, New {key}: {new_model_results[key]:.2f}, Difference: {new_model_results[key]-value:.2f}")

In [43]:
compare_baseline_to_new_results(baseline_results=baseline_results, new_model_results=model_1_results)

Baseline accuracy: 79.27, New accuracy: 78.74, Difference: -0.52
Baseline precision: 0.81, New precision: 0.79, Difference: -0.02
Baseline recall: 0.79, New recall: 0.79, Difference: -0.01
Baseline f1: 0.79, New f1: 0.78, Difference: -0.00


In [44]:
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [45]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [46]:
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [47]:
embed_weights

array([[ 0.00073165,  0.01504799, -0.03425454, ..., -0.04403539,
        -0.01042282,  0.0187644 ],
       [ 0.04135863, -0.03945084, -0.03811941, ...,  0.00464736,
         0.03163552,  0.02928299],
       [ 0.00684032,  0.05363132, -0.00241555, ..., -0.07082179,
        -0.04750703,  0.01448253],
       ...,
       [-0.03301444, -0.0052493 , -0.04209725, ...,  0.02028764,
         0.00308807,  0.02215792],
       [ 0.00692343,  0.05942352, -0.01975194, ..., -0.06199061,
        -0.01018393,  0.03510419],
       [-0.0372346 ,  0.06267188, -0.07451148, ..., -0.02367218,
        -0.08643329,  0.01742155]], dtype=float32)

In [48]:
import io

out_v = io.open("embedding_vectors.tsv", "w", encoding="utf-8")
out_m = io.open("embedding_metadata.tsv", "w", encoding="utf-8")

for num, word in enumerate(words_in_vocab):
    if num == 0: 
        continue
    vec = embed_weights[num]
    out_m.write(word + "\n")
    out_v.write("\t".join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()

In [49]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64, return_sequences=True)(x)
x = layers.LSTM(64)(x)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_2 = tf.keras.Model(inputs, outputs, name = 'model_2_lstm')

In [50]:
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [51]:
model_2.summary()

Model: "model_2_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 15, 64)            49408     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                      

In [52]:
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                     "LSTM")])

Saving TensorBoard log files to: model_logs/LSTM/20230926-125441
Epoch 1/5
215/215 [==============================] - 12s 20ms/step - loss: 0.2189 - accuracy: 0.9250 - val_loss: 0.5444 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 4s 16ms/step - loss: 0.1634 - accuracy: 0.9412 - val_loss: 0.6071 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.1282 - accuracy: 0.9520 - val_loss: 0.7574 - val_accuracy: 0.7808
Epoch 4/5
215/215 [==============================] - 3s 15ms/step - loss: 0.1076 - accuracy: 0.9584 - val_loss: 0.9243 - val_accuracy: 0.7848
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0815 - accuracy: 0.9635 - val_loss: 0.9809 - val_accuracy: 0.7808


In [53]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10]

24/24 [==============================] - 1s 6ms/step


((762, 1),
 array([[2.3730553e-03],
        [5.7842940e-01],
        [9.9988604e-01],
        [5.8599379e-02],
        [2.6671838e-05],
        [9.9545348e-01],
        [9.3895918e-01],
        [9.9989414e-01],
        [9.9978155e-01],
        [3.9918759e-01]], dtype=float32))

In [54]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [55]:
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 78.08398950131233,
 'precision': 0.7856875884556138,
 'recall': 0.7808398950131233,
 'f1': 0.777681907957685}

In [56]:
compare_baseline_to_new_results(baseline_results, model_2_results)

Baseline accuracy: 79.27, New accuracy: 78.08, Difference: -1.18
Baseline precision: 0.81, New precision: 0.79, Difference: -0.03
Baseline recall: 0.79, New recall: 0.78, Difference: -0.01
Baseline f1: 0.79, New f1: 0.78, Difference: -0.01


In [57]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.GRU(64, return_sequences=True)(x)
print(x.shape)
x = layers.LSTM(42, return_sequences=True)(x)
print(x.shape)
x = layers.GRU(99)(x)
print(x.shape)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_3 = tf.keras.Model(inputs, outputs, name='model_3_GRU')

(None, 15, 128)
(None, 15, 64)
(None, 15, 42)
(None, 99)


In [58]:
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [59]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 15, 64)            37248     
                                                                 
 lstm_2 (LSTM)               (None, 15, 42)            17976     
                                                                 
 gru_1 (GRU)                 (None, 99)                42471     
                                                       

In [60]:
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "GRU")])

Saving TensorBoard log files to: model_logs/GRU/20230926-125508
Epoch 1/5
215/215 [==============================] - 12s 25ms/step - loss: 0.1455 - accuracy: 0.9543 - val_loss: 0.6828 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0810 - accuracy: 0.9677 - val_loss: 0.9068 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0620 - accuracy: 0.9733 - val_loss: 1.3665 - val_accuracy: 0.7795
Epoch 4/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0563 - accuracy: 0.9736 - val_loss: 1.2454 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 4s 20ms/step - loss: 0.0498 - accuracy: 0.9762 - val_loss: 1.3617 - val_accuracy: 0.7651


In [61]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs.shape, model_3_pred_probs[:10]

24/24 [==============================] - 1s 8ms/step


((762, 1),
 array([[4.4146623e-02],
        [4.4326672e-01],
        [9.9999726e-01],
        [1.7379376e-01],
        [1.2176385e-05],
        [9.9953341e-01],
        [9.9765646e-01],
        [9.9999428e-01],
        [9.9999118e-01],
        [8.5444677e-01]], dtype=float32))

In [62]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [63]:
model_3_results = calculate_results(y_true=val_labels, 
                                    y_pred=model_3_preds)
model_3_results

{'accuracy': 76.50918635170603,
 'precision': 0.7669456967235844,
 'recall': 0.7650918635170604,
 'f1': 0.7627590065822708}

In [64]:
compare_baseline_to_new_results(baseline_results, model_3_results)

Baseline accuracy: 79.27, New accuracy: 76.51, Difference: -2.76
Baseline precision: 0.81, New precision: 0.77, Difference: -0.04
Baseline recall: 0.79, New recall: 0.77, Difference: -0.03
Baseline f1: 0.79, New f1: 0.76, Difference: -0.02


In [65]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_4 = tf.keras.Model(inputs, outputs, name='model_4_bidirectional')

In [66]:
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [67]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 15, 128)          98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              74496     
 nal)                                                            
                                             

In [68]:
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "bidirectional_RNN")])

Saving TensorBoard log files to: model_logs/bidirectional_RNN/20230926-125539
Epoch 1/5
215/215 [==============================] - 15s 30ms/step - loss: 0.1079 - accuracy: 0.9677 - val_loss: 0.8812 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 4s 21ms/step - loss: 0.0560 - accuracy: 0.9734 - val_loss: 1.0593 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 5s 21ms/step - loss: 0.0485 - accuracy: 0.9766 - val_loss: 1.4477 - val_accuracy: 0.7651
Epoch 4/5
215/215 [==============================] - 5s 22ms/step - loss: 0.0489 - accuracy: 0.9764 - val_loss: 1.1869 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 5s 22ms/step - loss: 0.0409 - accuracy: 0.9794 - val_loss: 1.5174 - val_accuracy: 0.7546


In [69]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 2s 11ms/step


array([[3.6026403e-02],
       [6.1233932e-01],
       [9.9996030e-01],
       [2.3535982e-01],
       [1.3480724e-05],
       [9.9995852e-01],
       [9.9928015e-01],
       [9.9997735e-01],
       [9.9996233e-01],
       [9.9939764e-01]], dtype=float32)

In [70]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [71]:
model_4_results = calculate_results(val_labels, model_4_preds)
model_4_results

{'accuracy': 75.45931758530183,
 'precision': 0.7541438250099668,
 'recall': 0.7545931758530183,
 'f1': 0.7540737922598122}

In [72]:
compare_baseline_to_new_results(baseline_results, model_4_results)

Baseline accuracy: 79.27, New accuracy: 75.46, Difference: -3.81
Baseline precision: 0.81, New precision: 0.75, Difference: -0.06
Baseline recall: 0.79, New recall: 0.75, Difference: -0.04
Baseline f1: 0.79, New f1: 0.75, Difference: -0.03


In [73]:
embedding_test = embedding(text_vectorizer(["this is a test sentence"]))
conv_1d = layers.Conv1D(filters = 32,
                        kernel_size = 5,
                        activation = 'relu',
                        padding = 'valid')
conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [74]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.01007083, -0.02885287, -0.01626282, ...,  0.03578598,
         -0.04404961, -0.01551962],
        [-0.00681981, -0.01829292, -0.02035785, ..., -0.01459057,
         -0.01062246,  0.06287219],
        [-0.04615291, -0.05724598, -0.04398639, ...,  0.01038231,
          0.00496608,  0.05433102],
        ...,
        [-0.01712118, -0.01241968, -0.02953283, ...,  0.00857792,
         -0.01626625,  0.03019948],
        [-0.01712118, -0.01241968, -0.02953283, ...,  0.00857792,
         -0.01626625,  0.03019948],
        [-0.01712118, -0.01241968, -0.02953283, ...,  0.00857792,
         -0.01626625,  0.03019948]]], dtype=float32)>

In [75]:
conv_1d_output

<tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
array([[[0.03984934, 0.06405033, 0.02558471, 0.00329244, 0.07815661,
         0.        , 0.07867042, 0.02180695, 0.01396454, 0.01175068,
         0.        , 0.00181683, 0.        , 0.03647531, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.01790879, 0.        , 0.        , 0.        ,
         0.01719212, 0.        , 0.03508216, 0.        , 0.08313186,
         0.        , 0.10407338],
        [0.0476682 , 0.        , 0.06267825, 0.02124223, 0.        ,
         0.08870694, 0.        , 0.03256815, 0.        , 0.        ,
         0.02284318, 0.        , 0.03112326, 0.06315568, 0.03262535,
         0.        , 0.02930659, 0.        , 0.        , 0.        ,
         0.        , 0.00402152, 0.        , 0.        , 0.        ,
         0.0122088 , 0.        , 0.        , 0.00280334, 0.        ,
         0.        , 0.        ],
        [0.00332462, 0.01618394, 0.03170469, 0.    

In [76]:
max_pool_output

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[0.0476682 , 0.06405033, 0.06267825, 0.02124223, 0.07815661,
        0.08870694, 0.07867042, 0.06665085, 0.03880931, 0.01175068,
        0.07918642, 0.02083474, 0.03112326, 0.06315568, 0.08318956,
        0.        , 0.02930659, 0.00444394, 0.05904168, 0.04732662,
        0.03087299, 0.0446954 , 0.06255046, 0.00548999, 0.02934461,
        0.01719212, 0.01220486, 0.03508216, 0.08780269, 0.08313186,
        0.        , 0.10407338]], dtype=float32)>

In [77]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters = 64,
                  kernel_size = 5,
                  strides = 1,
                  activation = 'relu',
                  padding = 'valid')(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_5 = tf.keras.Model(inputs, outputs, name = 'model_Conv_1D')

In [78]:
model_5.summary()

Model: "model_Conv_1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_6 (Dense)             (None, 1)               

In [79]:
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [80]:
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20230926-125617
Epoch 1/5
215/215 [==============================] - 4s 10ms/step - loss: 0.1350 - accuracy: 0.9566 - val_loss: 0.8141 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0793 - accuracy: 0.9714 - val_loss: 0.9858 - val_accuracy: 0.7703
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0647 - accuracy: 0.9752 - val_loss: 1.0520 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0572 - accuracy: 0.9775 - val_loss: 1.1164 - val_accuracy: 0.7598
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0521 - accuracy: 0.9775 - val_loss: 1.1705 - val_accuracy: 0.7703


In [81]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[1.1362345e-01],
       [6.5586007e-01],
       [9.9994874e-01],
       [5.8081564e-02],
       [8.7707861e-08],
       [9.9612266e-01],
       [9.5123720e-01],
       [9.9998474e-01],
       [9.9999893e-01],
       [8.4479058e-01]], dtype=float32)

In [82]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [83]:
model_5_results = calculate_results(y_true=val_labels, y_pred=model_5_preds)
model_5_results

{'accuracy': 77.03412073490814,
 'precision': 0.7720778836459699,
 'recall': 0.7703412073490814,
 'f1': 0.7681933440908485}

In [84]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [85]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_samples = embed(["The quick brown fox jumps over the lazy dog.", "I am a sentence for which I would like to get its embedding"])

print(embed_samples)

tf.Tensor(
[[-0.03133018 -0.06338634 -0.01607499 ... -0.0324278  -0.04575738
   0.05370456]
 [ 0.05080859 -0.0165243   0.01573778 ...  0.00976658  0.03170118
   0.01788118]], shape=(2, 512), dtype=float32)


In [86]:
embed_samples[0].shape

TensorShape([512])

In [87]:
sentance_encoded_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape = [],
                                        dtype = 'string',
                                        trainable = False,
                                        name = 'USE')

In [88]:
model_6 = tf.keras.Sequential([sentance_encoded_layer, 
                               layers.Dense(64, activation = 'relu'), #New Layer added
                               layers.Dense(1, activation='sigmoid', name = 'Output_layer')],
                               name = 'model_6_USE')

In [89]:
model_6.compile(loss = 'binary_crossentropy',
                optimizer= 'adam',
                metrics = ['accuracy'])

In [90]:
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_7 (Dense)             (None, 64)                32832     
                                                                 
 Output_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [91]:
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "tf_hub_sentance_encoder")])

Saving TensorBoard log files to: model_logs/tf_hub_sentance_encoder/20230926-125650
Epoch 1/5
215/215 [==============================] - 9s 26ms/step - loss: 0.5032 - accuracy: 0.7866 - val_loss: 0.4483 - val_accuracy: 0.8005
Epoch 2/5
215/215 [==============================] - 5s 22ms/step - loss: 0.4153 - accuracy: 0.8146 - val_loss: 0.4377 - val_accuracy: 0.8136
Epoch 3/5
215/215 [==============================] - 5s 23ms/step - loss: 0.4010 - accuracy: 0.8205 - val_loss: 0.4338 - val_accuracy: 0.8123
Epoch 4/5
215/215 [==============================] - 5s 23ms/step - loss: 0.3941 - accuracy: 0.8248 - val_loss: 0.4295 - val_accuracy: 0.8150
Epoch 5/5
215/215 [==============================] - 5s 22ms/step - loss: 0.3883 - accuracy: 0.8272 - val_loss: 0.4315 - val_accuracy: 0.8150


In [92]:
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

24/24 [==============================] - 1s 21ms/step


array([[0.14376776],
       [0.74101067],
       [0.98478705],
       [0.17871137],
       [0.747965  ],
       [0.6611063 ],
       [0.9774665 ],
       [0.97439176],
       [0.9131896 ],
       [0.08313901]], dtype=float32)

In [93]:
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [94]:
model_6_results = calculate_results(y_true=val_labels, y_pred=model_6_preds)
model_6_results

{'accuracy': 81.49606299212599,
 'precision': 0.8172549323109193,
 'recall': 0.8149606299212598,
 'f1': 0.8134357776936025}

In [95]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [96]:
train_10_percent = train_df_shuffled[['text', 'target']].sample(frac=0.1, random_state=42)
train_sentences_10_percent = train_10_percent['text'].to_list()
train_labels_10_percent = train_10_percent['target'].to_list()

In [97]:
len(train_labels_10_percent), len(train_sentences_10_percent)

(761, 761)

In [98]:
train_10_percent['target'].value_counts()

target
0    413
1    348
Name: count, dtype: int64

In [99]:
train_df_shuffled['target'].value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [100]:
model_7 = tf.keras.models.clone_model(model_6)

model_7.compile(loss="binary_crossentropy",
                optimizer='adam',
                metrics=["accuracy"])

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run Identity: Dst tensor is not initialized. [Op:Identity]

In [ ]:
model_7.summary()

In [ ]:
model_7_history = model_7.fit(x=train_sentences_10_percent,
                              y=train_labels_10_percent,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "10_percent_tf_hub_sentence_encoder")])